In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataver2-train-test/__results__.html
/kaggle/input/dataver2-train-test/data_ver2.h5
/kaggle/input/dataver2-train-test/__notebook__.ipynb
/kaggle/input/dataver2-train-test/__output__.json
/kaggle/input/dataver2-train-test/custom.css
/kaggle/input/dataver3/dataver3.h5
/kaggle/input/dataver3/__results__.html
/kaggle/input/dataver3/__notebook__.ipynb
/kaggle/input/dataver3/__output__.json
/kaggle/input/dataver3/custom.css
/kaggle/input/dataver4/__results__.html
/kaggle/input/dataver4/__huggingface_repos__.json
/kaggle/input/dataver4/dataver4.h5
/kaggle/input/dataver4/__notebook__.ipynb
/kaggle/input/dataver4/__output__.json
/kaggle/input/dataver4/custom.css
/kaggle/input/v4-kgat-data/kgat_data/val.txt
/kaggle/input/v4-kgat-data/kgat_data/relation_list.txt
/kaggle/input/v4-kgat-data/kgat_data/item_list.txt
/kaggle/input/v4-kgat-data/kgat_data/user_list.txt
/kaggle/input/v4-kgat-data/kgat_data/test.txt
/kaggle/input/v4-kgat-data/kgat_data/kg_final.txt
/kaggle/input/v4-kgat-data

In [2]:
import pandas as pd

def filter_user_and_course(user_course, min_interactions=5):
    """
    Xóa các user và course có số lượng tương tác ít hơn min_interactions.
    
    Args:
        user_course (pd.DataFrame): DataFrame chứa các cột ['user_id', 'course_id'].
        min_interactions (int): Số lượng tương tác tối thiểu để giữ user hoặc course.

    Returns:
        pd.DataFrame: DataFrame sau khi lọc.
    """
    while True:
        # Đếm số lượng tương tác của mỗi user và mỗi course
        user_counts = user_course['user_id'].value_counts()
        course_counts = user_course['course_id'].value_counts()

        # Lấy user và course đạt yêu cầu
        valid_users = user_counts[user_counts >= min_interactions].index
        valid_courses = course_counts[course_counts >= min_interactions].index

        # Lọc dữ liệu
        filtered = user_course[
            (user_course['user_id'].isin(valid_users)) &
            (user_course['course_id'].isin(valid_courses))
        ]

        # Nếu không có gì thay đổi nữa thì dừng
        if len(filtered) == len(user_course):
            break
        user_course = filtered

    return filtered


In [3]:
path = [
    "",
    "/kaggle/input/dataver2-train-test/data_ver2.h5",
    "/kaggle/input/dataver3/dataver3.h5",
    "/kaggle/input/dataver4/dataver4.h5",
    "/kaggle/input/dataver5/data_ver5.h5"
]

In [4]:
df = pd.read_hdf(path[2],key="df")


In [5]:
user = pd.read_csv("/kaggle/input/random-user/sample_user.csv")

In [6]:
# user = user.sample(n=3000, random_state=42)

In [7]:
df_t = df[df['user_id'].isin(user['user_id'])]

In [8]:
df.user_id.nunique()

367103

In [9]:
df_t.user_id.nunique()

24090

In [10]:
df_t.course_id.nunique()

2712

In [11]:
len(user)

24581

In [12]:
filtered_data = filter_user_and_course(df_t, min_interactions=5)
filtered_data.user_id.nunique(),filtered_data.course_id.nunique()


(24068, 2539)

In [13]:
filtered_data.to_hdf("data_sample.h5",key="df")

/tmp/ipykernel_19/1862063921.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['user_id', 'course_id', 'course_name', 'course_prerequisites',
       'course_about', 'course_field', 'video_id', 'video_ccid',
       'video_duration', 'exercise_id', 'exercise_num_problem', 'concept_id',
       'concept_prerequisite', 'concept_name', 'concept_field', 'school_id',
       'school_name', 'school_about', 'school_motto', 'teacher_id',
       'teacher_name', 'teacher_about', 'teacher_job_title',
       'teacher_org_name', 'watch_id', 'total_watch_time', 'actual_watch_time',
       'doing_exercise_id', 'doing_num_correct'],
      dtype='object')]

  filtered_data.to_hdf("data_sample.h5",key="df")


In [14]:
!rm -rf /kaggle/working/sample

In [15]:
!pip install gitpython
import git
git.Repo.clone_from('https://github.com/Briandy23/courserec.git', '/kaggle/working/sample')
%cd /kaggle/working/

/kaggle/working


In [16]:
!python /kaggle/working/sample/src/UPGPR/preprocess_moocX.py \
--dataset /kaggle/working/data_sample.h5 \
        --save_dir /kaggle/working/sample/data/moocx 

Reading relations from /kaggle/working/data_sample.h5
Reading relation: course-video
Reading relation: course-exercise
Reading relation: course-field
Reading relation: course-concept
Reading relation: course-school
Reading relation: course-teacher
Reading relation: user-course
Removing concepts in less than 1 courses
Removing users enrolled in less than 1 courses
Extracting entities
Number of users: 24068
Number of courses: 2539
Number of teachers: 8454
Number of schools: 398
Number of concepts: 62974
Number of videos: 54972
Number of exercises: 53311
Number of fields: 77
Saving entities
Saving /kaggle/working/sample/data/moocx/users.txt
Saving /kaggle/working/sample/data/moocx/courses.txt
Saving /kaggle/working/sample/data/moocx/teachers.txt
Saving /kaggle/working/sample/data/moocx/schools.txt
Saving /kaggle/working/sample/data/moocx/concepts.txt
Saving /kaggle/working/sample/data/moocx/videos.txt
Saving /kaggle/working/sample/data/moocx/exercises.txt
Saving /kaggle/working/sample/dat

In [17]:
!python /kaggle/working/sample/src/UPGPR/make_dataset.py --config /kaggle/working/sample/config/UPGPR/moocx.json

{'data_dir': '/kaggle/working/sample/data/moocx', 'tmp_dir': '/kaggle/working/tmp/moocx', 'seed': 23, 'ratio': 0.8, 'ratio_validation': 0.5, 'data_file': 'enrolments.txt', 'use_user_relations': False, 'use_entity_relations': False, 'use_wandb': False, 'wandb_project_name': 'project_name', 'wandb_run_name': 'run_name'}
Loading dataset from folder: /kaggle/working/sample/data/moocx
Load user of size 24068
Load item of size 2539
Load school of size 398
Load teacher of size 8454
Load concept of size 62974
Load video of size 54972
Load exercise of size 53311
Load field of size 77
Load item_concept of size 2538
Load item_school of size 2539
Load item_teacher of size 2539
Load item_video of size 2538
Load item_exercise of size 2538
Load item_field of size 2539
Load interactions of size 286952
Creating knowledge graph from dataset...
Load entities...
Total 206793 nodes.
Load enrolments...
Total 573904 enrolment edges.
Load course knowledge item_concept...
Total 1386 item_concept edges.
Load co

In [18]:
!python /kaggle/working/sample/src/UPGPR/train_transe_model.py --config /kaggle/working/sample/config/UPGPR/moocx.json

[INFO]  {'data_dir': '/kaggle/working/sample/data/moocx', 'tmp_dir': '/kaggle/working/tmp/moocx', 'name': 'train_transe_model', 'seed': 23, 'gpu': '0', 'epochs': 30, 'min_epochs': 29, 'batch_size': 32, 'lr': 0.001, 'patience': 30, 'use_user_relations': False, 'use_entity_relations': False, 'weight_decay': 0, 'l2_lambda': 0, 'max_grad_norm': 5.0, 'embed_size': 100, 'num_neg_samples': 5, 'steps_per_checkpoint': 200, 'use_wandb': False, 'wandb_project_name': 'mooc_testing', 'wandb_run_name': '200_epoch_embed', 'device': device(type='cuda'), 'log_dir': '/kaggle/working/tmp/moocx/train_transe_model'}
[INFO]  Parameters:['enroll', 'item_concept', 'item_school', 'item_teacher', 'item_video', 'item_exercise', 'item_field', 'user.weight', 'item.weight', 'school.weight', 'teacher.weight', 'concept.weight', 'video.weight', 'exercise.weight', 'field.weight', 'enroll_bias.weight', 'item_concept_bias.weight', 'item_school_bias.weight', 'item_teacher_bias.weight', 'item_video_bias.weight', 'item_exer

In [19]:
!python /kaggle/working/sample/src/UPGPR/train_agent.py --config /kaggle/working/sample/config/UPGPR/moocx.json

[INFO]  {'data_dir': '/kaggle/working/sample/data/moocx', 'tmp_dir': '/kaggle/working/tmp/moocx', 'name': 'train_agent', 'seed': 23, 'gpu': '0', 'epochs': 50, 'min_epochs': 49, 'batch_size': 32, 'lr': 0.001, 'patience': 10, 'max_acts': 250, 'max_path_len': 3, 'gamma': 0.99, 'ent_weight': 0.001, 'act_dropout': 0.5, 'state_history': 2, 'hidden': [512, 256], 'use_pattern': False, 'reward': 'binary_train', 'modified_policy': False, 'sum_prob': False, 'use_wandb': False, 'wandb_project_name': 'project_name', 'wandb_run_name': 'run_name', 'device': device(type='cuda'), 'log_dir': '/kaggle/working/tmp/moocx/train_agent'}
Load embedding: /kaggle/working/tmp/moocx/transe_embed.pkl
[INFO]  Parameters:['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias', 'actor.weight', 'actor.bias', 'critic.weight', 'critic.bias']
[INFO]  epoch/step=1/100 | loss=0.33557 | ploss=-0.00127 | vloss=0.34462 | entropy=-7.77599 | reward=0.10750
[INFO]  epoch/step=1/200 | loss=0.28799 | ploss=0.00863 | vloss=0.28692 | entrop

In [20]:
!python /kaggle/working/sample/src/UPGPR/test_agent.py --config /kaggle/working/sample/config/UPGPR/moocx.json

Predicting paths...
Load embedding: /kaggle/working/tmp/moocx/transe_embed.pkl
24080it [20:42, 19.38it/s]
Load embedding: /kaggle/working/tmp/moocx/transe_embed.pkl
{'self_loop_user-->enroll_item-->enroll_user-->enroll_item-->': 3126847, 'self_loop_user-->enroll_item-->item_teacher_teacher-->item_teacher_item-->': 1244, 'self_loop_user-->enroll_item-->item_field_field-->item_field_item-->': 25115, 'self_loop_user-->enroll_item-->item_school_school-->item_school_item-->': 29842}
Min courses to consider user valid=10.000 |  Compute metrics for all users=True

NDCG=9.663 |  Recall=17.285 | HR=17.285 | Precision=1.728 | HR@1=3.787 | HR@3=8.759 | HR@5=11.971 | Invalid users=93

NDCG=9.854 |  Recall=17.521 | HR=17.521 | Precision=1.769 | HR@1=3.893 | HR@3=9.024 | HR@5=12.228 | Computed for all users.



In [21]:
!python /kaggle/working/sample/src/UPGPR/test_agentX.py --config /kaggle/working/sample/config/UPGPR/moocx.json

Predicting paths...
Load embedding: /kaggle/working/tmp/moocx/transe_embed.pkl
24080it [19:40, 20.40it/s]
Load embedding: /kaggle/working/tmp/moocx/transe_embed.pkl
{'self_loop_user-->enroll_item-->enroll_user-->enroll_item-->': 3125937, 'self_loop_user-->enroll_item-->item_teacher_teacher-->item_teacher_item-->': 1194, 'self_loop_user-->enroll_item-->item_school_school-->item_school_item-->': 29592, 'self_loop_user-->enroll_item-->item_field_field-->item_field_item-->': 24861}
Min courses to consider user valid=10 | Compute metrics for all users=True

Metrics @ K=5 (Valid users):
NDCG=7.930 | Recall=11.874 | HR=11.874 | Precision=2.375 | HR@1=3.798 | HR@3=8.785 | MAP=6.634 | F1=3.958 | Invalid users=83

Metrics @ K=5 (All users):
NDCG=8.103 | Recall=12.107 | HR=12.107 | Precision=2.421 | HR@1=3.889 | HR@3=9.008 | MAP=6.785 | F1=4.036

Metrics @ K=10 (Valid users):
NDCG=9.617 | Recall=17.090 | HR=17.090 | Precision=1.709 | HR@1=3.798 | HR@3=8.785 | MAP=7.330 | F1=3.107 | Invalid users=